In [57]:
!pip install fastapi
!pip install uvicorn
!pip install pydantic
!pip install scikit-learn
!pip install pyngrok

In [67]:
from pydantic import BaseModel
import uvicorn
from uvicorn import Config, Server
from fastapi import FastAPI
import pickle
from sklearn.preprocessing import StandardScaler
import asyncio
import numpy as np
import pandas as pd
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
from pyngrok import ngrok

app = FastAPI()

model_path = "/content/house_predictor_model (1).pkl"
scaler_path = "/content/House_data_scaler (1).pkl"

class House(BaseModel):
  propertyType: str
  size: int
  rooms: int
  bathrooms: int
  floor: int
  hasLift: bool
  ParkingSpace: bool
  city: str
  status: int

def load_model(model_path=model_path,scaler_path=scaler_path):
  with open(model_path, 'rb') as file:
      model = pickle.load(file)
  with open(scaler_path, 'rb') as file:
      scaler = pickle.load(file)
  return model, scaler

def preprocessing(data, scaler):
  city_mapping = {"Lisboa": 0.500292, "Cascais": 0.202730, "Oeiras": 0.084173, "Loures": 0.072127, "Amadora": 0.047452, "Odivelas": 0.047379, "Vila Franca de Xira": 0.045846}
  prop_type_mapping = {'flat': 0.7649291867425901, "chalet": 0.1800992845670901, "duplex": 0.026646225726383414, "studio": 0.01657176230106585, "penthouse": 0.009198423127463863, "countryHouse": 0.0025551175354066287}

  default_city = np.mean(list(city_mapping.values()))
  default_prop = np.mean(list(prop_type_mapping.values()))

  data["city"] = data["city"].map(city_mapping).fillna(default_city)
  data["propertyType"] = data["propertyType"].map(prop_type_mapping).fillna(default_prop)

  data["hasLift"] = int(data["hasLift"])
  data["ParkingSpace"] = int(data["ParkingSpace"])

  data[["size", "rooms", "bathrooms", "floor", ]] = scaler.transform(data[["size", "rooms", "bathrooms", "floor", ]])
  return data

@app.post("/")
def make_prediction(house: House):
  model, scaler = load_model(model_path=model_path,scaler_path=scaler_path)
  data = pd.DataFrame(house.dict())
  preprocessed_data = preprocessing(data, scaler)
  prediction = model.predict(preprocessed_data)
  return {"prediction": prediction.tolist()}

if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app)

INFO:     Started server process [321]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('127.0.0.1', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-31' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:67> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-53-fcb94147e6aa>", line 57, in <cell line: 56>
    uvicorn.run(app, host="0.0.0.0", port=8000)
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 575, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn